A hydrostatic, inviscid, adiabatic, and moist system in isentropic coordinates 
==

In [ ]:
from datetime import timedelta
import gridtools as gt
import numpy as np
import sympl
import tasmania as taz

# For interactive plotting
from IPython import display
%matplotlib nbagg

Load the computational grid and the initial state
--

In [ ]:
grid, states = taz.load_netcdf_dataset(
    '../tests/baseline_datasets/isentropic_moist_sedimentation.nc')
state = states[0]

Initialize the microphysical parameterizations
--

In [ ]:
# The component calculating the microphysical sources/sinks 
# prescribed by the Kessler scheme; neglect the evaporation of
# rain in the subcloud layers
a = sympl.DataArray(0.1, attrs={'units': 'g kg^-1'})
kessler = taz.Kessler(grid, pressure_on_interface_levels=True,
                      rain_evaporation_on=False, backend=gt.mode.NUMPY,
                      autoconversion_threshold=a)

# The component calculating the raindrop fall velocity
rfv = taz.RaindropFallVelocity(grid, backend=gt.mode.NUMPY)

# The component performing the saturation adjustment 
# as prescribed by the Kessler scheme
saturation = taz.SaturationAdjustmentKessler(grid, pressure_on_interface_levels=True,
                                             backend=gt.mode.NUMPY)

Instantiate the dynamical core
--

In [ ]:
# Adopt the flux form of the forward Euler time integration method,
# with the horizontal numerical fluxes calculated via the MacCormack scheme;
# the sedimentation flux is discretized with the help of the second-order upwind scheme
dycore = taz.IsentropicDynamicalCore(grid, moist_on=True,
                                     time_integration_scheme='forward_euler',
                                     horizontal_flux_scheme='maccormack',
                                     horizontal_boundary_type='relaxed',
                                     smooth_on=True, smooth_type='first_order',
                                     smooth_coeff=0.20, smooth_at_every_stage=True,
                                     adiabatic_flow=True, sedimentation_on=True,
                                     sedimentation_flux_type='second_order_upwind',
                                     sedimentation_substeps=2,
                                     raindrop_fall_velocity_diagnostic=rfv,
                                     backend=gt.mode.NUMPY, dtype=np.float64)

Set up the full model
--

In [ ]:
# Cast the physical components into PhysicsComponentComposite wrappers
physics_before_dynamics = taz.PhysicsComponentComposite('parallel_splitting', kessler)
physics_after_dynamics  = taz.PhysicsComponentComposite('parallel_splitting', saturation)

# Build the model
model = taz.Model(dycore, physics_before_dynamics, physics_after_dynamics)

Create a monitor for interactive visualization
---------------------------------------------------------------

In [ ]:
# The artist and its collaborators generating the left subplot
subplot1 = taz.Plot2d(grid, plot_function=taz.make_contourf_xz, 
                      field_to_plot='mass_fraction_of_cloud_liquid_water_in_air', level=0,
                      plot_properties={'fontsize': 16, 
                                       'title_left': 'Cloud liquid water [g kg$^{-1}$]',
                                       'x_label': '$x$ [km]', 'x_lim': [0, 500],
                                       'y_label': '$z$ [km]', 'y_lim': [0, 14]},
                      plot_function_kwargs={'fontsize': 16, 
                                            'x_factor': 1e-3, 'z_factor': 1e-3,
                                            'field_factor': 1e3,
                                            'cmap_name': 'Blues', 'cbar_on': True, 
                                            'cbar_levels': 18, 'cbar_ticks_step': 4, 
                                            'cbar_orientation': 'horizontal'})

# The artist generating the right subplot
coll1 = taz.Plot1d(grid, plot_function=taz.plot_horizontal_profile, 
                   field_to_plot='precipitation', levels={1: 0, 2: 0},
                   plot_function_kwargs={'fontsize': 16, 'x_factor': 1e-3,
                                         'linecolor': 'blue', 'linewidth': 1.5,
                                         'legend_label': 'Precipitation [mm h$^{-1}$]'})
coll2 = taz.Plot1d(grid, plot_function=taz.plot_horizontal_profile, 
                   field_to_plot='accumulated_precipitation', levels={1: 0, 2: 0},
                   plot_function_kwargs={'fontsize': 16, 'x_factor': 1e-3,
                                         'linecolor': 'red', 'linewidth': 1.5,
                                         'legend_label': 'Accumulated precipitation [mm]'})
subplot2 = taz.PlotsOverlapper((coll1, coll2), fontsize=16, 
                               plot_properties={'fontsize': 16,
                                                'x_label': '$x$ [km]', 'x_lim': [0, 500],
                                                'y_lim': [0, 1], 'grid_on': True,
                                                'legend_on': True, 'legend_loc': 'best'})

# The monitor encompassing and coordinating the two artists
plot_monitor = taz.SubplotsAssembler(nrows=1, ncols=2, artists=(subplot1, subplot2), 
                                     interactive=False, figsize=(12, 7), fontsize=16, 
                                     tight_layout=True)

Integrate the model forward in time
--

In [ ]:
# Simulation settings
timestep = timedelta(seconds=10)
niter    = 2160

for i in range(niter):
    # Integrate the modul one timestep forward in time 
    state_new = model(state, timestep)
    state.update(state_new)
    
    if (i+1) % 60 == 0:
        # Plot the solution
        subplot1.plot_properties['title_right'] = str((i+1)*timestep)
        subplot2.plot_properties['title_right'] = str((i+1)*timestep)
        fig = plot_monitor.store((state, (state, state)), show=False)
        display.clear_output(wait=True)
        display.display(fig)

print('Simulation successfully completed. HOORAY!')

Results validation
--

In [ ]:
for name in state.keys():
    if name != 'time':
        assert np.allclose(states[-1][name].values, state[name].values), \
               'Variable {} not validated.'.format(name)
print('Results validated. HORRAY! HOORAY!')